<a href="https://colab.research.google.com/github/vinrus/progetto-visione/blob/main/src/neuralnetwork/classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Librerie


In [1]:
import os
import keras
import matplotlib.style as style
import numpy as np
from PIL import Image

from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.callbacks import Callback
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Input
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from keras import models, layers
import tensorflow as tf 

from random import randint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting
import tables #gestione di set di dati gerarchici

#style.use('seaborn-whitegrid')


### Connessione a google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Constanti


In [5]:
pathDataset='/content/drive/MyDrive/Visione e Percezione/progetto-visione/dataset' #VINCENZO
#pathDataset='../../assets/dataset/' LOCALE
pathSaveModel='/content/drive/MyDrive/Visione e Percezione/progetto-visione/assets/models/'
#pathSaveModel='../../assets/models/' LOCALE
epoche=3

### Check dataset

1. Costruiamo un dizionario 'label' per memorizzare i nomi dei gesti che andremo ad indentificare
2. Costruiamo un dizionario 'reverseLabel' per memorizzare a quale gesto è associato un determinato identificatore
   // Label inserite all'interno del dataset nella prima cartella 00 che usero per i tutti i test


In [6]:
label = dict()
reverseLabel = dict()
count = 0
for j in os.listdir(pathDataset + '/00'):
    if not j.startswith('.'):
        label[j] = count
        reverseLabel[count] = j
        count = count + 1

print("Label=")
print(label)
print("ReverseLabel=")
print(reverseLabel)


Label=
{'06_index': 0, '10_down': 1, '08_palm_moved': 2, '05_thumb': 3, '01_palm': 4, '07_ok': 5, '04_fist_moved': 6, '02_l': 7, '03_fist': 8, '09_c': 9}
ReverseLabel=
{0: '06_index', 1: '10_down', 2: '08_palm_moved', 3: '05_thumb', 4: '01_palm', 5: '07_ok', 6: '04_fist_moved', 7: '02_l', 8: '03_fist', 9: '09_c'}


### Dataset

lettura dei dati memorizzandoli nell'array 'datas'
memorizziamo il classificatore numerico per ogni immagine in 'datasNum'


In [ ]:

datas = []
datasNum = []

countImage = 0  # count delle immagini che ci sono nel dataset

for i in range(0, 10):  # ciclo su 10 cartelle del primo livello
    for j in os.listdir(pathDataset+'0' + str(i) + '/'):
         if not j.startswith('.'):  # evito le cartelle nascoste 
            count = 0  # conta le imagini di un gesto
            for k in os.listdir(pathDataset+'0' + str(i) + '/' + j + '/'): # ciclo le immagini
                img = Image.open(pathDataset + '0' + str(i) + '/' + j + '/' + k).convert('L') # leggo è converto in scala di grigi
                img = img.resize((224, 224))
                arr = np.array(img)
                datas.append(arr) 
                count = count + 1

            #     img = img.resize((224, 224)) ##img.resize((320, 120))#
            #     datas.append(np.array(img))
            #     count = count + 1
            
            datasNum.append(np.full((count, 1), label[j]))
            countImage = countImage + count


datasNum = np.array(datasNum)
datasNum = datasNum.reshape(countImage, 1)

print("[DEBUG] numero di immagini esaminate: " + str(countImage))

In [ ]:
#Visualizzazione delle immagini
plt.figure(figsize=(20,20)) # specifying the overall grid size

for i in range(0, 10):
    # for x in range(25):
        plt.subplot(5,5,i+1)
        plt.title(reverseLabel[i])
        plt.imshow(datas[i*200])

plt.show()

In [ ]:
#ridimensione del formato dell'immagini cosi da avere comprese tra 0 e 1
datas = np.array(datas, dtype = 'float32')
datas = datas.reshape(len(datas), 224, 224, 1)
datas /= 255

#conversione di datasNum in one-hot
datasNum = np.array(datasNum)
datasNum = datasNum.reshape(len(datas), 1)
datasNum = to_categorical(datasNum)

### Split dei dati tramite Skleanr 
divisione del dataset in 2 parti 80 - 20 (train) e 50 - 50 (validate)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(datas, datasNum, test_size=0.2)
xValidation, xTest, yValidation, yTest = train_test_split(datas, datasNum, test_size=0.5)

# Modello tramite keras: 

Inzio ad aggiungere al modello di keras un filtro 5*5 con passo 2. 
Aggiungiamo una sequenza di livelli di convoluzione seguiti da un raggruppamento massimo fino a un'immagine piccola
In fine aggiungiamo uno strato di softmax con 10 neuroni

Di seguito adattiamo il modello 

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(224, 224,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.summary()

Allenamentodellarete--con10epocheiniziali--

In [ ]:
history = model.fit(xTrain, yTrain, 
        epochs=epoche,
        batch_size=64, 
        verbose=1, 
        validation_data=(xValidation, yValidation))

### Plot History

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epoche) 

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Preparazione della 'Matrice di convoluzione'
y_pred=model.predict(xTest) 
y_pred=np.argmax(yValidation, axis=1)
y_test=np.argmax(yTest, axis=1)
cm = confusion_matrix(y_test, y_pred)
# print(cm)

In [ ]:
#Plot Matrice di convoluzione 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label)
fig, ax = plt.subplots(figsize=(10,10))
plt.title("Confusion Matrix")
disp = disp.plot(ax=ax)
plt.show()

In [ ]:
# valutazione delle previsioni della rete  
valutazione = model.predict(xTest, batch_size=64) 
print(classification_report(yTest.argmax(axis=1),
                            valutazione.argmax(axis=1), 
                            target_names=label))


### Save Model

In [ ]:
#model.save(pathSaveModel) save all
model.save(pathSaveModel+'model.h5')

# Creazione del modello con VGG16

In [ ]:
# Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(
    weights='imagenet',
    include_top=False,
)
model_vgg16_conv.summary()


In [ ]:
# Crezionde dellimmaginie del formato (224x224x3) per li nostro input della rete
imageInput = Input(shape=(224, 224, 3), name='imageInput')

# makes the layers non-trainable
for layer in model_vgg16_conv.layers:
    layer.trainable = False

# Uso il generatore di modello:
outputVgg16Conv = model_vgg16_conv(imageInput)

# Add the fully-connected layers 
x = Flatten(name='flatten')(outputVgg16Conv)
x = Dense(10, activation='softmax', name='predictions')(x)  # here the 2 indicates binary (3 or more is multiclass)

# Creazionde del nostro modello 
my_model = Model(imageInput, x)

# In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

In [ ]:
# ottimizzazione del modello: 
baseModel = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
optimizer = tf.keras.optimizers.Adam() # Adam is like a gradient descent (way to find parameters)

In [ ]:
# Add top layer
x = baseModel.output
x = Flatten()(x)
x = Dropout(0.5)(x)

predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=baseModel.input, outputs=predictions)

# Train top layer
for layer in baseModel.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]

model.summary()

In [ ]:
history = model.fit(xTrain[:500], 
        yTrain[:500], 
        epochs=4, 
        batch_size=64,
        validation_data=(xValidation[:500], yValidation[:500]),
        verbose=1
        )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epoche) 

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#https://www.kaggle.com/code/benenharrington/hand-gesture-recognition-database-with-cnn/notebook
#perdopo:https://techvidvan.com/tutorials/hand-gesture-recognition-tensorflow-opencv/